# 7장 범주형 데이터 이진분류 경진대회

# 경진대회 이해하기

# 라이브러리 및 데이터 불러오기

In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np

In [ ]:
# 필요한 데이터 불러오기 - 패스 설정하기
data_path="/kaggle/input/cat-in-the-dat/"

In [ ]:
# 데이터 지정하기

train = pd.read_csv(data_path + 'train.csv', index_col = 'id')
test = pd.read_csv(data_path + 'test.csv', index_col = 'id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col = 'id')

index_col은 불러올 DataFrame의 인덱스를 지정하는 파라미터.

지정하지 않으면 인덱스열을 추가해서 데이터프레임을 생성함.

In [ ]:
train.shape, test.shape

In [ ]:
train.head().T   # transpose해주는 옵션

In [ ]:
submission.head()

# 탐색적 데이터 분석하기

탐색적 데이터 분석의 시작은 피처의 특성을 파악하는 것부터라고 할 수 있다. 
* 피처 타입
* 결측값 분석
* 고유값 분석
* 입력된 값의 형태

등등에 대해 알아볼 필요가 있다.

In [ ]:
# 데이터 타입 확인하기
summary=pd.DataFrame(train.dtypes, columns=['데이터 타입'])
print(summary)

피처 요약표 만들기

In [ ]:
# index 열 추가
summary = summary.reset_index()
print(summary)

In [ ]:
# 열 이름 변경
summary = summary.rename(columns = {'index':'피처'})
print(summary)

In [ ]:
# 결측값 파악하기
summary['결측값 개수'] = train.isnull().sum().values
print(summary)

In [ ]:
summary['고윳값 개수'] = train.nunique().values
summary['첫 번째 값'] = train.loc[0].values
summary['두 번째 값'] = train.loc[1].values
summary['세 번재 값'] = train.loc[2].values

summary

함수를 만들어 한번에 처리하는 것이 효율적이다.

In [ ]:
def resumetable(df):
    print(f'데이터셋 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns = {'index':'피처'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윳값 개수'] = df.nunique().values
    summary['첫 번째 값'] = df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values
    summary['세 번재 값'] = df.loc[2].values
    
    return summary

resumetable(train)
    

In [ ]:
train.head()

In [ ]:
for i in range(3):
    feature = 'ord_'+str(i)
    print(f'{feature} 고유값: {train[feature].unique()}')

In [ ]:
for i in range(3,6):
    feature = 'ord_'+str(i)
    print(f'{feature} 고유값: {train[feature].unique()}')

In [ ]:
print('day 고유값:', train['day'].unique())
print('day 고유값:', train['month'].unique())
print('day 고유값:', train['target'].unique())

## 데이터 시각화

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

### 타깃값의 분포

In [ ]:
mpl.rc('font', size=15) # 폰트의 크기를 지정
plt.figure(figsize=(7, 6))

ax = sns.countplot(x='target', data=train)
ax.set_title("Target Distribution")

In [ ]:
mpl.rc('font', size=15) # 폰트의 크기를 지정
plt.figure(figsize=(7, 6))

ax = sns.countplot(x='target', data=train)
ax.set_title("Target Distribution")
print(ax.patches)

.patches 메쏘드는 ax축을 구성하는 그래프 도형 객체 모두를 닮은 리스트이다. 

In [ ]:
print(ax.patches)

위에서 그린 사각형에 대한 정보를 담고 있다. 이를 이용하면 다양한 시각화 효과를 만들어 낼 수 있다.

In [ ]:
rectangle_01 = ax.patches[0]
rectangle_02 = ax.patches[1]

# 사각형에 대한 정보보기
print('0_사각형의 높이:', rectangle_01.get_height())
print('1_사각형의 높이:', rectangle_02.get_height())
print('0_사각형의 너비:', rectangle_01.get_width())
print('1_사각형의 너비:', rectangle_02.get_width())
print('0_사각형의 x좌표:', rectangle_01.get_x())
print('1_사각형의 x좌표:', rectangle_02.get_x())

이러한 정보를 이용하여 막대 그래프 위에 정보를 표시해보자.

In [ ]:
def write_percent(ax, total_size):
    for patch in ax.patches:
        height = patch.get_height()
        width = patch.get_width()
        left_coord = patch.get_x()
        percent = height/total_size * 100
        
        
        # (x, y) 좌표에 텍스트 입력
        ax.text(x=left_coord+ width/2.0,
               y=height + total_size*0.001,
               s=f'{percent:1.1f}%',
               ha='center')
# 그래프 그리기
plt.figure(figsize=(7, 6))

ax = sns.countplot(x='target', data=train)
write_percent(ax, len(train))
ax.set_title("Target Distribution")

---

## 이진 피처 분포 살펴보기

In [ ]:
train.head()

In [ ]:
for i in range(5):
    feature = 'bin_'+str(i)
    print(f'{feature} 고유값: {train[feature].unique()}')

5개의 그래프를 그리기 위해 gridspec 라이브러리를 이용한다.

In [ ]:
import matplotlib.gridspec as gridspec

mpl.rc('font', size=12)
grid = gridspec.GridSpec(3, 2) # 3행 2열로 배치
plt.figure(figsize=(10, 16))   # 전체 그래프 크기 지정하기
plt.subplots_adjust(wspace=0.4, hspace=0.3)  # 서브플롯 간 좌우 /상하 여백 지정

# 서브 플롯 그리기
bin_features = summary['피처'][0:5]
# 원래 교재에는 bin_features = ['bin_0','bin_1','bin_2','bin_3','bin_4']로 되어 있던 것을 수정한 코드

for idx, feature in enumerate(bin_features):
    ax = plt.subplot(grid[idx])
    
    sns.countplot(x=feature,
                  data=train,
                  hue='target',
                  palette='pastel',
                  ax = ax)
    ax.set_title(f'{feature} Distribution by Target')
    write_percent(ax, len(train))

---

## 명목형 피처 분포 살펴보기

In [ ]:
train.head().T

### 단계 01. 교차 분석표 만들기

교차표는 범주형 데이터 2개를 분석하는데 이용한다. 명목형 변수인 nom_0~nom_9 중에서 의미를 알 수 없는 데이터가 있는 부분을 제외하고 nom_0 ~nom_4번까지만 살펴보자.

In [ ]:
pd.crosstab(train['nom_0'], train['target'])

In [ ]:
# 백분율로 나타내기
crosstab = pd.crosstab(train['nom_0'], train['target'], normalize='index')*100
crosstab

그래프를 그릴 때 nom_0의 정보를 이용하기 위해서 인덱스를 만들어서 열을 만들어준다.

In [ ]:
crosstab = crosstab.reset_index()
crosstab

이 부분도 재사용을 위해 함수로 만들어 둔다.

In [ ]:
def get_crosstab(df, feature):
    crosstab = pd.crosstab(df[feature], df['target'], normalize='index')*100
    crosstab = crosstab.reset_index()
    return crosstab

In [ ]:
crosstab=get_crosstab(train, 'nom_0')
crosstab

In [ ]:
crosstab[1]

### 단계 02. 포인트플롯 함수 생성하기

In [ ]:
def plot_pointplot(ax, feature, crosstab):
    ax2 = ax.twinx()  # X축을 공유하는 그래프 그리기
    
    ax2 = sns.pointplot(x=feature, y=1, data=crosstab,
                       order=crosstab[feature].values,
                       color='black',
                       legend=False)
    ax2.set_ylim(crosstab[1].min()-5, crosstab[1].max()*1.1)
    ax2.set_ylabel('Target 1 Ratio(%)')

In [ ]:
def plot_cat_dist_with_true_ratio(df, features, num_rows, num_cols, size=(15, 20)):
    
    plt.figure(figsize=size)
    grid = gridspec.GridSpec(num_rows, num_cols) # 3행 2열로 배치
    plt.subplots_adjust(wspace=0.4, hspace=0.3)  # 서브플롯 간 좌우 /상하 여백 지정

    for idx, feature in enumerate(features):
        ax = plt.subplot(grid[idx])
        crosstab = get_crosstab(df, feature)

        sns.countplot(x=feature,
                      data=df,
                      order=crosstab[feature].values,
                      color='skyblue',
                      ax = ax)
        write_percent(ax, len(train))
        
        plot_pointplot(ax, feature, crosstab)        
        
        ax.set_title(f'{feature} Distribution')
        

In [ ]:
nom_features = summary['피처'][5:10]
plot_cat_dist_with_true_ratio(train, nom_features, num_rows=3, num_cols=2)

### 순서형 피처 분포

In [ ]:
ord_features = ['ord_0','ord_1','ord_2','ord_3']
plot_cat_dist_with_true_ratio(train, ord_features, num_rows=2, num_cols=2, size=(15, 12))

순서형 변수의 경우 순서대로 보여주는 것이 필요하다. 

In [ ]:
from pandas.api.types import CategoricalDtype

ord_1_value = ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster']
ord_2_value = ['Freezing', 'Cold', 'Warm', 'Hot', 'Boiling Hot', 'Lava Hot']

ord_1_dtype = CategoricalDtype(categories=ord_1_value, ordered =True)
ord_2_dtype = CategoricalDtype(categories=ord_2_value, ordered =True)

train['ord_1'] = train['ord_1'].astype(ord_1_dtype)
train['ord_2'] = train['ord_2'].astype(ord_2_dtype)

In [ ]:
plot_cat_dist_with_true_ratio(train, ord_features, num_rows=2, num_cols=2, size=(15, 12))

In [ ]:
plot_cat_dist_with_true_ratio(train, ['ord_4', 'ord_5'], num_rows=2, num_cols=1, size=(15, 12))

In [ ]:
date_features=['day', 'month']
plot_cat_dist_with_true_ratio(train, date_features, num_rows=2, num_cols=1, size=(15, 12))

# 베이스라인 모델 생성하기

## 피처 엔지니어링

* 머신러닝 모델은 문자를 인식하지 못함
* 문자를 숫자로 치환 (인코딩이라고 부르는 과정)
* 훈련용 데이터와 테스트 데이터에 동일한 인코딩을 적용하기 위해 데이터를 합친 후 인코딩
* 테스트 데이터에는 타깃값이 없으므로 타깃값을 제거
* 피처와 타깃값은 따로 분리해서 모델링해야 함

In [ ]:
# 데이터 합치기
all_data = pd.concat([train, test])

# target 제거
all_data = all_data.drop('target', axis=1)
all_data

# 원-핫 인코딩

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder() # 원-핫 인코더 생성

all_data_encoded = encoder.fit_transform(all_data)

## 데이터 분리하기

생각도 못한 멋진 방법으로 데이터를 분리

In [ ]:
num_train = len(train)

X_train = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]

y = train['target']

훈련 데이터의 일부를 검증 데이터로 나누기

stratify: 사회학적 개념으로 '수평적 지위 집단으로 나눈다'라는 의미, 즉 파라미터로 지정한 값을 각 그룹에 공평하게 분배한다는 의미
<br> 타깃값이 편중되지 않도록 배분해주면 된다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                     test_size = 0.1,
                                                     stratify = y,
                                                     random_state = 10)

## 모델 훈련하기

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_model = LogisticRegression(max_iter = 1000, random_state=42)

max_iter: 모델의 회귀 계수를 업데이트하는 반복 횟수, 모델 훈련 시 회귀 계수를 업데이트하면서 훈련하는데 이때 업데이트를 몇 번 하는지 결정해줌

In [ ]:
logistic_model.fit(X_train, y_train)

In [ ]:
# 모델 성능 검증
logistic_model.predict_proba(X_valid)

In [ ]:
logistic_model.predict(X_valid)

In [ ]:
y_valid_preds = logistic_model.predict_proba(X_valid)[:,1]

모델 성능을 검증하기 위해서 ROC AUC 점수를 이용함. 이후 추가로 정리할 것

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC: {roc_auc:.4f}')

## 예측 및 결과 제출

In [ ]:
y_preds = logistic_model.predict_proba(X_test)[:, 1]

In [ ]:
# 제출 파일 생셩
submission['target'] = y_preds

In [ ]:
submission.to_csv('submission.csv')

# 성능 개선하기 1

## <span style="color:blue"> 주안점</span>

* 피처 맞춤 인코딩
* 피처 스케일링: 피처의 값 범위를 일치시켜 특정 피처의 영향을 통제하는 방법
* 하이퍼파라미터 최적화


In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np

# 필요한 데이터 불러오기 - 패스 설정하기
data_path="/kaggle/input/cat-in-the-dat/"

# 데이터 지정하기

train = pd.read_csv(data_path + 'train.csv', index_col = 'id')
test = pd.read_csv(data_path + 'test.csv', index_col = 'id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col = 'id')

### 피처 맞춤형 인코딩하기

1. 원본 데이터 중 이진 데이터에 대해 피처 인코딩
2. 순서형 피처 인코딩
3. 명목형, 날짜 피처 인코딩
4. 모든 작업 후 데이터 합치기

In [ ]:
# 데이터 합치기
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1)

In [ ]:
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

In [ ]:
# 순서형 피처 인코딩하기

# 고유값 확인하기
for i in range(6):
    feature = 'ord_'+str(i)
    print(f'{feature} 고유값: {train[feature].unique()}')

In [ ]:
ord1dict={'Novice':0, 'Contributor':1,'Expert':2,'Master':3,'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2, 'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3', 'ord_4', 'ord_5']

ord_encoder = OrdinalEncoder()

all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])

for feature, categories in zip(ord_345, ord_encoder.categories_):
    print(feature)
    print(categories)

In [ ]:
nom_features = ['nom_' + str(i) for i in range(10)]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

encoded_nom_matrix


In [ ]:
all_data = all_data.drop(nom_features, axis=1)

In [ ]:
date_features = ['day', 'month']

encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])
all_data = all_data.drop(date_features, axis=1)

encoded_date_matrix

In [ ]:
# 피처 스케일링

from sklearn.preprocessing import MinMaxScaler

ord_featues = ['ord_'+str(i) for i in range(6)]

all_data[ord_featues] = MinMaxScaler().fit_transform(all_data[ord_featues])

In [ ]:
# 피처 합치기
from scipy import sparse

In [ ]:
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                               encoded_nom_matrix,
                               encoded_date_matrix],
                              format='csr')

all_data_sprs

### 데이터 나누기

In [ ]:
num_train = len(train)
num_train

In [ ]:
X_train = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y=train['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, 
                                                     test_size = 0.1,
                                                     stratify=y,
                                                     random_state=10)

### 하이퍼 파라미터 최적화

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#로지스틱 회귀 모형 생성
logistic_model = LogisticRegression()

# 하이퍼 파라미터 목록
lr_params = {'C':[0.1, 0.125, 0.2], 'max_iter':[800, 900, 1000],
            'solver':['liblinear'], 'random_state':[42]}

gridsearch_logistic_model = GridSearchCV(estimator=logistic_model,
                                        param_grid=lr_params,
                                        scoring='roc_auc',
                                        cv=5)

gridsearch_logistic_model.fit(X_train, y_train)

print('최적 하이퍼파라미터:', gridsearch_logistic_model.best_params_)

In [ ]:
y_valid_preds = gridsearch_logistic_model.predict_proba(X_valid)[:, 1]

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC: {roc_auc: .4f}')

In [ ]:
y_preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_test)[:,1]

In [ ]:
submission['target'] = y_preds

submission.to_csv('submission.csv')

# 성능 개선하기 2